In [186]:
import geemap
import ee
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/4AFbHX5o9dQStHlinwULHFCAFQSCctMMgiQlfGEUPZb5USCjYQON1a8

Successfully saved authorization token.


In [187]:
#Display simple map
Map = geemap.Map () 
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [188]:
#choose which basemap to use... default is google basemap
Map.add_basemap("Google Maps")

In [189]:
Map.centerObject(ROI, 6)

In [190]:
# defining ROI
xmin,ymin,xmax,ymax = [122.7328125,72.187559318, 129.47779664, 73.97591048]

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [191]:
#import Landsat 8 images between june and August 2018 falling within ROI
images2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
         .filterBounds(ROI).filterDate('2018-06-01', '2018-08-31')

In [192]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').clip(ROI);
occurrence = gsw.select('occurrence')
#Map.addLayer(occurrence,'Water Occurrence (1984-2015)', True, 0.5);

In [193]:
#Visualization params
VIS_OCCURRENCE = {
        'min': 0,
        'max': 100,
        'palette': ['red', 'blue']}


In [194]:
occ = occurrence.updateMask(occurrence.divide(100))

In [195]:
Map.addLayer(occ,VIS_OCCURRENCE , 'occurrence', True, 0.5)

In [196]:
# --------Mask pixels with clouds and cloud shadows ---------

# Surface reflectance products come with a 'pixel_qa' band

# create function to mask clouds, cloud shadows, snow

def maskClouds(image):
  pixel_qa = image.select('pixel_qa')
  return image.updateMask(pixel_qa.eq(322)) # keep clear (0) and water (1) pixels  

# apply the masking function on the image collection
images2018_masked = images2018.map(maskClouds)

# visualize the first image in the collection, pre- and post- mask
visParams = {
    'bands': ['B4','B3','B2'], 
    'min': 0, 
    'max': 3000}

Map.addLayer(ee.Image(images2018_masked.first()), visParams, 'cloudsmasked_2018', False)
Map.addLayer(ee.Image(images2018.first()), visParams, 'original_2018', False)

In [197]:
#Calculate ndvi

def getNDVI(img):
  return img.addBands(img.normalizedDifference(ee.List(['B5', 'B4'])).rename('NDVI'))

In [198]:
#Visualization params for NDVI
ndviPalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];
               

In [199]:
# "map" the NDVI band over image collection
ndvi2018 = images2018_masked.map(getNDVI);

In [200]:
# print one image to see the band is now there
#print(ee.Image(ndvi2018.first()));

In [201]:
#for each pixel, select the "best" set of bands from available images
#based on the maximum NDVI/greenness

#composite the image based on max NDVI
#this code creats a single image composit based on max NDVI value
composite = ndvi2018.qualityMosaic('NDVI').clip(ROI);
#print(composite);

In [202]:
#Visualize true color composite and compare with ndvi
Map.addLayer(composite.select('NDVI'),
            {'min':0, 'max': 1, 'palette': ndviPalette}, 'ndvi_2018');
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'TC_composite_2018', False);



In [203]:
#/* color: #206f1b */
MESH = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.35823708277854, 72.34141113708156],
                  [126.36141281825218, 72.3417235695464],
                  [126.36029701930198, 72.34232238347865],
                  [126.36055451136741, 72.34349391909498],
                  [126.36192780238304, 72.34474347414152],
                  [126.36544686061058, 72.34758068744891],
                  [126.3627861092678, 72.34791904122308],
                  [126.3613269875637, 72.34692998946569],
                  [126.35815125209007, 72.34487363120185],
                  [126.35737877589378, 72.34318151695544],
                  [126.35797959071311, 72.34164546193192]]]),
            {
              "landcover": 0,
              "system:index": "0"
            })]),
    
#/* color: #777957 */
DSH = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.4707300848755, 72.37197869128515],
                  [126.47167422244874, 72.37307034878854],
                  [126.47373415897218, 72.37369412368697],
                  [126.47442080447999, 72.37436985570798],
                  [126.47450663516847, 72.37494160936514],
                  [126.47613741824952, 72.37556532014617],
                  [126.47922732303468, 72.37574723176496],
                  [126.47957064578858, 72.37663077662478],
                  [126.47965647647706, 72.37769617073704],
                  [126.47871233890382, 72.37785207683953],
                  [126.47665240238038, 72.37748829385617],
                  [126.47484995792237, 72.37686464896741],
                  [126.47459246585694, 72.3760330892071],
                  [126.47339083621827, 72.37520149145446],
                  [126.47115923831788, 72.37494160936514],
                  [126.46944262454835, 72.37431787721229],
                  [126.46841265628663, 72.37377209404646],
                  [126.467039365271, 72.37343422007675],
                  [126.46738268802491, 72.3726284953036],
                  [126.46875597904054, 72.37226460791064],
                  [126.46987177799073, 72.37182273486958]]]),
            {
              "landcover": 1,
              "system:index": "0"
            })]),
#/* color: #44a243 */
GSSH = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.05537711238799, 72.33130464887066],
                  [126.05743704891142, 72.33187775500286],
                  [126.06061278438506, 72.33289367164502],
                  [126.05975447750029, 72.33362301282],
                  [126.05872450923857, 72.33430023209671],
                  [126.057179556846, 72.33568063971356],
                  [126.05778037166533, 72.33633173913508],
                  [126.05915366268096, 72.33672238763559],
                  [126.0604411230081, 72.33732136575811],
                  [126.05803786373076, 72.33708698535644],
                  [126.05709372615752, 72.33667030165212],
                  [126.05503378963408, 72.33586294989362],
                  [126.05589209651885, 72.33489928973768],
                  [126.05743704891142, 72.3339095317351],
                  [126.05743704891142, 72.3333364894062],
                  [126.0549479589456, 72.33312810591589],
                  [126.05391799068389, 72.33273738045798],
                  [126.0560637578958, 72.33211220232445],
                  [126.05400382137236, 72.33219035076237],
                  [126.05211554622588, 72.33263318558964],
                  [126.0505705938333, 72.3326071367796],
                  [126.04851065730986, 72.33237269581615],
                  [126.04893981075224, 72.33172145511523]]]),
            {
              "landcover": 2,
              "system:index": "0"
            })]),
#/* color: #208c73 */
WSMT = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.18354687582521, 72.34271199307435],
                  [126.16672406088381, 72.34895918005088],
                  [126.17805371176271, 72.35353909042307],
                  [126.18183026205568, 72.35947044550994],
                  [126.20002636801271, 72.36300752765767],
                  [126.21650586020021, 72.36820787166168],
                  [126.2103260506299, 72.37215914015222],
                  [126.19865307699709, 72.37642138933003],
                  [126.17839703451662, 72.36373566520824],
                  [126.15264782797365, 72.36040679879385],
                  [126.12930188070803, 72.35697326814393],
                  [126.12748331699612, 72.34191148320875],
                  [126.13400644932034, 72.33909942732501]]]),
            {
              "landcover": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.28679308215636, 72.34755890644053],
                  [126.28735909353762, 72.34394858537996],
                  [126.28418335806398, 72.34085043813356],
                  [126.28006348501711, 72.33840279008963],
                  [126.27396950613527, 72.33600690163075],
                  [126.26392731558352, 72.33262087062303],
                  [126.26452813040285, 72.33100577299925],
                  [126.2701929558423, 72.32983343520331],
                  [126.27491364370852, 72.33118812988873],
                  [126.27525696646242, 72.33389721762978],
                  [126.2861145485547, 72.33784290954587],
                  [126.2927235115674, 72.33906681218092],
                  [126.29761586081057, 72.34294630121268],
                  [126.30010495077639, 72.34586188402123],
                  [126.29993328939943, 72.34908930702201],
                  [126.2952984322217, 72.35015631315225],
                  [126.28671536337404, 72.34914135755041]]]),
            {
              "landcover": 3,
              "system:index": "1"
            })]),

#/* color: #bb7936 */
DT = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.28336796652346, 72.35515219388569],
                  [126.28405461203127, 72.3546058361216],
                  [126.28276715170412, 72.35379927805779],
                  [126.28345379721193, 72.35343501432916],
                  [126.28551373373537, 72.35265442468165],
                  [126.28345379721193, 72.3519258441825],
                  [126.2813938606885, 72.35143143368676],
                  [126.28379711996584, 72.35111916751697],
                  [126.28791699301271, 72.35171767297665],
                  [126.29109272848635, 72.35260238418319],
                  [126.29169354330568, 72.35351307145531],
                  [126.29298100363283, 72.3544237132271],
                  [126.29212269674807, 72.35517821051353],
                  [126.29032025229006, 72.35595869207886],
                  [126.28783116232424, 72.35629689037683],
                  [126.2846554268506, 72.356270875346],
                  [126.28276715170412, 72.35556845547437]]]),
            {
              "landcover": 4,
              "system:index": "0"
            })]),
#/* color: #942a6e */
PC = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.38377893797805, 72.38036275645679],
                  [126.37905825011184, 72.37919358944515],
                  [126.37210596434524, 72.37815426681607],
                  [126.36472452513625, 72.37703692878844],
                  [126.36463869444778, 72.37641326843324],
                  [126.36781442992141, 72.37555570055106],
                  [126.36884439818313, 72.3745421591434],
                  [126.37450922362258, 72.37365851289519],
                  [126.38257730833938, 72.37503594277631],
                  [126.38935793272903, 72.37677707290415],
                  [126.39656771056106, 72.37810229912654],
                  [126.40231836668899, 72.37872590160728],
                  [126.40480745665481, 72.37971322850187],
                  [126.39957178465774, 72.38267488780417],
                  [126.39648187987258, 72.38418151191519],
                  [126.39176119200637, 72.38309052071033]]]),
            {
              "landcover": 5,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.34554136626174, 72.39603532817543],
                  [126.33670080534866, 72.39468544765738],
                  [126.3376449429219, 72.39354316277854],
                  [126.34270895354202, 72.39219309721355],
                  [126.34785879485061, 72.391206447446],
                  [126.35378111235549, 72.3906871365919],
                  [126.35584104887893, 72.39673618814476]]]),
            {
              "landcover": 5,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.49588692021258, 72.37979703217226],
                  [126.50369751286395, 72.37875774400658],
                  [126.50481331181415, 72.37743256551163],
                  [126.50361168217547, 72.37675694718216],
                  [126.5006076080788, 72.37602933016397],
                  [126.49648773503192, 72.37556156100644],
                  [126.49726021122821, 72.37447005288898],
                  [126.49871933293231, 72.3735084319857],
                  [126.4994918091286, 72.37224785197371],
                  [126.50266754460223, 72.37177998563625],
                  [126.50738823246844, 72.37175399270942],
                  [126.51236641240008, 72.37232582852768],
                  [126.51511299443133, 72.37417117615693],
                  [126.51399719548114, 72.37554856724712],
                  [126.51073562931903, 72.3778093417636],
                  [126.50961983036883, 72.37957618840323],
                  [126.50575744938739, 72.38170656945132],
                  [126.50009262394794, 72.38204428988358],
                  [126.49597275090106, 72.38108306910773]]]),
            {
              "landcover": 5,
              "system:index": "2"
            })]),
#/* color: #f7ff22 */
SAND = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[125.9393329285878, 72.32821098107804],
                  [125.95340916149796, 72.32456290210976],
                  [125.96147724621476, 72.32174817163929],
                  [125.9722919129628, 72.321591785002],
                  [125.97383686535538, 72.3252404576103],
                  [125.96662708752335, 72.32909683310189],
                  [125.95358082287491, 72.33138942695575],
                  [125.94276615612686, 72.33196253042578],
                  [125.93864628307999, 72.33102471537731]]]),
            {
              "landcover": 6,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.30515148762308, 72.2454813011039],
                  [126.32214596394144, 72.24239260830437],
                  [126.3391404402598, 72.24375379124689],
                  [126.33725216511331, 72.24945919001514],
                  [126.33459141377054, 72.25435185686689],
                  [126.32695248249613, 72.25558135931857],
                  [126.29837086323343, 72.2554244061495],
                  [126.27236416462503, 72.25521513316741]]]),
            {
              "landcover": 6,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.74804543819322, 72.43653388561057],
                  [126.86426019039048, 72.43927912483272],
                  [126.9033989843358, 72.44269714365163],
                  [126.9092354711522, 72.45853590462625],
                  [126.87318658199204, 72.45455163757799],
                  [126.84606408443345, 72.45449988808504],
                  [126.87146996822251, 72.45889806726517],
                  [126.90528725948228, 72.46153646219032],
                  [126.8817696508397, 72.461329543147],
                  [126.8429741796483, 72.47208619863937],
                  [126.74907540645493, 72.4576045960133],
                  [126.70152520503892, 72.44616629158514]]]),
            {
              "landcover": 6,
              "system:index": "2"
            })]),

#/* color: #202cff */
WATER = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.30263551782177, 72.33350106305649],
                  [126.31465181420849, 72.34199058984001],
                  [126.33267625878857, 72.34740515637034],
                  [126.3373111159663, 72.35313035252399],
                  [126.33061632226513, 72.35240179104848],
                  [126.31997331689404, 72.34823802392259],
                  [126.30933031152294, 72.34376091347104],
                  [126.30023225854443, 72.33813693847239]]]),
            {
              "landcover": 7,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.17938264916943, 72.32651898359269],
                  [126.18830904077099, 72.33214654005026],
                  [126.16805299829052, 72.3319381429719]]]),
            {
              "landcover": 7,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.16530641625927, 72.29783320113414],
                  [126.1780093581538, 72.29825077171384],
                  [126.18075594018505, 72.30190410760038],
                  [126.16805299829052, 72.30200847790066]]]),
            {
              "landcover": 7,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.28375276635693, 72.31171231200321],
                  [126.28478273461864, 72.30712189282187],
                  [126.29817232202099, 72.31098209516833],
                  [126.2930224807124, 72.31744871472337],
                  [126.28409608911083, 72.3181786731416]]]),
            {
              "landcover": 7,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[126.62105658949595, 72.5010579358483],
                  [126.61762336195689, 72.46985462326207],
                  [126.6564188331483, 72.4719227331526],
                  [126.66465857924204, 72.48938882853601],
                  [126.68182471693736, 72.50374179698161],
                  [126.66843512953501, 72.51148147011862],
                  [126.64920905531626, 72.50807642264233]]]),
            {
              "landcover": 7,
              "system:index": "4"
            })]);

In [204]:
#merge classes
#classNames = ee.FeatureCollection(MESH).merge(DSH).merge(GSSH).merge(WSMT).merge(DT).merger(PC).merge(SAND).merge(WATER)
#print (classNames)